In [61]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random

#OpenWeatherMap API Key
api_key = "26fea83d1259ec5c651394906a953b54"

In [94]:
#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 5): 
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    if cityToCheck not in cityList:
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Column for city number
raw_cities_df['City Number'] = raw_cities_df.index
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

In [95]:
raw_cities_df.head()
#print(raw_cities_df['City Number'])

,City Number,City Name
0,0,half moon bay
1,1,kenai
2,2,ballina
3,3,ponta delgada
4,4,vostok


In [ ]:
settings = {"units": "imperial","appid": api_key}
from time import sleep

#Get current weather for each city in cityList
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")

summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

for city in raw_cities_df["City Name"]:
    sleep(2)
    #print("Processing:... "+ city['City Number'])
    print("Processing:... "+ city ) 
    current_city= ow.get_current(city, **settings)
    sleep(1)
    #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
    #summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
    summary_output = current_city(*summary)
    #weather_data.append(current_city)
    print(str(summary_output))
    
    


Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']
Processing:... half moon bay
('Half Moon Bay', 37.46, -122.43, 57.7, 72, 13.87, 1)
Processing:... kenai
('Kenai', 60.55, -151.26, 10.09, 100, 6.93, 90)
Processing:... ballina
('Ballina', 54.11, -9.15, 36.82, 100, 10.42, 76)
Processing:... ponta delgada
('Ponta Delgada', 37.73, -25.67, 62.6, 67, 10.29, 40)
